In [1]:
import os
import re
import pandas as pd
import numpy as np

# pip install PyGithub
from github import Github

# First create a Github instance:
g = Github("jmarrec", "<web_password>")

# Find NREL repos that may have a gemspec file

## Filter on repos that have 'Ruby' as a language

In [2]:
repo_names = []
ruby_repo_names = []
repo = None
for repo in g.get_organization('NREL').get_repos():
    repo_name = repo.name
    # print(repo_name)
    repo_names.append(repo_name)
    if 'Ruby' in repo.get_languages():
        ruby_repo_names.append(repo_name)

# Loop on all repos that have ruby, parse *.gemspec file

In [3]:
RE_VERSION = re.compile(r'^\s+\w+\.add_(?P<type>(?:.*?_)?dependency)\s+[\'\"](?P<name>.*?)[\'\"]\s*,\s*[\'\"](?P<version>.*?)[\'\"]')

In [4]:
# List of branches that are open and aim to bump dependencies
# If not specified here, will use develop

bump_branches = {
 'NREL/OpenStudio-workflow-gem': 'Bump_deps',
 'NREL/OpenStudio-measure-tester-gem': 'upgrade-dependencies',
 'NREL/openstudio-common-measures-gem': 'feature/13-openstudio-3.x',
}


gemspecs = {}
for ruby_repo_name in ruby_repo_names:
    repo = g.get_user('NREL').get_repo(ruby_repo_name)
    repo_full_name = repo.full_name
    
    if (repo_full_name in bump_branches):
        b = repo.get_branch(bump_branches[repo_full_name])
        print("For {r}, getting branch {b} at {s}".format(
          r=repo_full_name, b=bump_branches[repo_full_name], s=b.commit.sha))
        content_files = repo.get_contents('.', ref=b.commit.sha)
    else:
        content_files = repo.get_contents('.')
        
    for content_file in content_files:
    
        if os.path.splitext(content_file.name)[1] == '.gemspec':
            gemspecs[repo_full_name] = []
            content = content_file.decoded_content.decode()

            for line in content.splitlines():
                m = RE_VERSION.search(line)
                if m:
                    d = m.groupdict()
                    gemspecs[repo_full_name].append(d)

For NREL/OpenStudio-workflow-gem, getting branch Bump_deps at e191192cebe86df57b323bfb21e16b4e2b5ac2bc
For NREL/OpenStudio-measure-tester-gem, getting branch upgrade-dependencies at f0aaf5ad0383105070162be58663c3d46cb7e24e
For NREL/openstudio-common-measures-gem, getting branch feature/13-openstudio-3.x at fbcd76d8b426b538acc874f0d308e969507c29c4


# Analyze versions

In [5]:
empty_gemspecs = [k for k, v in gemspecs.items() if not v]

In [6]:
for k, v in gemspecs.items():
    for x in v:
        x.update({'gem': k})
        #x.pop('gem')
        
gemspecs_list = []

for k, v in gemspecs.items():
    for x in v:
        gemspecs_list.append(x)

In [7]:
df = pd.DataFrame(gemspecs_list)[['gem', 'name', 'version', 'type']]

In [8]:
# df.pivot(index='name', columns='gem', values='version')

df_piv = df.pivot(index='gem', columns='name', values=['version', 'type'])

## Visualize

In [9]:
def check_versions(col):
    """
    col is a pd.Series for a single gem used in our projects
    
    Return:
    * True if no problems
    * False if problems: found more than one required version
    """
    return (col[col.notnull()].nunique() <= 1)

In [10]:
def color_by_type(val):
    if val == 'dependency':
        return 'color: red'
    elif val == 'development_dependency':
        return 'color: orange'
    else:
        return ''

def color_by_dep_type(data):
    return df_piv['type'].loc[data.index, data.columns].applymap(color_by_type)

In [11]:
(df_piv['version'].loc[:, ~df_piv['version'].apply(check_versions)].fillna('').style
   .apply(color_by_dep_type, axis=None)
   .set_caption("Gem versions. Red = add_dependency, orange = add_development_dependency (add_runtime_dependency not handled).\nList of branches used (develop otherwise):\n{d}".format(d=bump_branches))
)

name,bundler,faraday,nokogiri,openstudio-extension,openstudio-workflow,openstudio_measure_tester,parallel,public_suffix,rake,roo,rspec,rubocop,rubocop-checkstyle_formatter
gem,,,,,,,,,,,,,
NREL/OpenStudio-analysis-gem,,~> 0.14,~> 1.8.2,,,,,,~> 12.3.1,~> 2.7.1,,,
NREL/OpenStudio-aws-gem,,,,,,,,,~> 12.3,,,,
NREL/OpenStudio-measure-tester-gem,~> 2.1,,,,,,,,~> 13.0,,~> 3.9,~> 0.80.1,~> 0.4.0
NREL/OpenStudio-workflow-gem,~> 2.1,,,,,,~> 1.19.1,~> 4.0.3,,,~> 3.9,~> 0.80.1,~> 0.4.0
NREL/bricr,~> 1.17.1,,,,,,~> 1.12,,,,,,
NREL/dencity-gem,~> 1.10,,,,,,,,~> 10.0,,,,
NREL/dencity-scripts,~> 1.7,,,,,,,,~> 10.0,,,,
NREL/openstudio-common-measures-gem,~> 2.1,,,~> 0.2.0,,,~> 1.12.0,,~> 12.3,,~> 3.7,~> 0.54.0,
NREL/openstudio-extension-gem,~> 2.1,,,,~> 1.3.4,~> 0.2.0,~> 1.12.0,,~> 12.3,,~> 3.7,~> 0.54.0,


## Save to HTML for sharing

In [12]:
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])


def getStyles():
    styles = [
        hover(),
        dict(selector="tr:nth-child(2n+1)", props=[('background', '#f5f5f5')]),
        dict(selector="td", props=[("text-align", "center")]),
        dict(selector="caption", props=[("caption-side", "bottom"),
                                        ("color", "grey")])
    ]
    return styles



html = (df_piv['version'].loc[:, ~df_piv['version'].apply(check_versions)].fillna('').style
   .set_table_attributes('style="border:1px solid black;'
                                     'border-collapse:collapse;"')
   .set_properties(**{'border': '1px solid black',
                                  'border-collapse': 'collapse',
                                  'border-spacing': '0px'})
   .apply(color_by_dep_type, axis=None)
   .set_table_styles(getStyles())
   .set_caption("Gem versions. Red = add_dependency, orange = add_development_dependency (add_runtime_dependency not handled).\nList of branches used (develop otherwise):\n{d}".format(d=bump_branches))
).render()

with open('result.html', 'w') as f:
    f.write(html)